# Bert를 사용한 문장 간 관계 분류. 사용자 vocab을 사용한

2개의 문장을 주고, 3개의 카테고리 중의 하나로 분류한다.

연관, 중립, 상반

copy from https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/blob/master/7.PRETRAIN_METHOD/7.2.2.bert_finetune_KorNLI.ipynb

# 필요 라이브러리 설치

In [ ]:
!pip install transformers==3.0.2
!pip install sentencepiece

In [ ]:
!pip install konlpy

# 셋업

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf

In [ ]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

SEQ_LENGTH = 128
BERT_MODEL_NAME = 'bert-base-multilingual-cased'
CUSTOM_VOCAB_FILE = 'custom_vocab.txt'

# 데이터

## 데이터 다운로드

In [ ]:
!git clone https://github.com/kakaobrain/KorNLUDatasets

fatal: destination path 'KorNLUDatasets' already exists and is not an empty directory.


In [ ]:
!wc ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv

  550153  8595590 78486224 ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv


## 데이터 로딩

In [ ]:
df = pd.read_csv("KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv", delimiter = '\t', quoting = 3)

In [ ]:
df.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


## 카테고리 인덱스 만들기

In [ ]:
df.gold_label = df.gold_label.astype('category')

In [ ]:
df['category'] = df.gold_label.cat.codes

In [ ]:
df.head()

,sentence1,sentence2,gold_label,category
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral,2
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction,0
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment,1
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral,2
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment,1


In [ ]:
category_names = list(df.gold_label.cat.categories)
print(category_names)

['contradiction', 'entailment', 'neutral']


## 데이터 섞기

In [ ]:
df = df.sample(frac=1).reset_index(drop=True) 

df.head()

,sentence1,sentence2,gold_label,category
0,다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다.,한 남자가 스파이더맨 침대보를 망토로 사용하여 슈퍼 스파이디가 될 수 있다!,neutral,2
1,회색 셔츠를 입은 남자가 사교 모임을 위해 사운드보드를 작동시키고 있다.,DJ가 음악을 연주하고 있다.,neutral,2
2,자전거를 탄 한 남자가 모터 크로스 행사 중에 흙을 토한다.,폭주족이 이벤트에서 이기고 있다.,neutral,2
3,한 남자가 배트맨 복장을 하고 포즈를 취하고 있다.,남자가 변장을 하고 있다.,entailment,1
4,깨끗한 컵 용기에 액체를 섞고 있는 한 여성과 함께 실험실처럼 보이는 대머리 남자,여자가 남자가 필로폰을 만드는 것을 보고 있다.,neutral,2


## 필요 입출력 값 준비

In [ ]:
sentences1 = df.sentence1.values.copy().astype(np.str)
sentences2 = df.sentence2.values.copy().astype(np.str)
labels = df.category.values.copy().astype(np.int)

In [ ]:
print(sentences1.shape)
print(sentences2.shape)
print(labels.shape)

(550152,)
(550152,)
(550152,)


필요 시, 실습 시간 관계로 전체 중에 일부 만 사용한다.

In [ ]:
SHORT_COUNT = 100000
sentences1 = sentences1[:SHORT_COUNT]
sentences2 = sentences2[:SHORT_COUNT]
labels = labels[:SHORT_COUNT]

## Vocab 파일 만들기

### 전체 문장을 파일로 저장

In [ ]:
all_sentence = []
all_sentence.extend(sentences1)
all_sentence.extend(sentences2)

In [ ]:
print(len(all_sentence))
print(all_sentence[0])

1100304
다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다.


### vocab builder 생성

In [ ]:
from konlpy.tag import Okt
import collections
from collections import OrderedDict

BERT_PREFIX = "##"

class KonlpyVocabMaker():

  def __init__(self, texts):
    self._tokens = []
    self._tokenize(texts)

  # texts = '하늘이 푸른가요? 나는 푸른색이 좋아요'
  # return ['하늘', '##이', '푸른가요', '?', '나', '##는', '푸른색', '##이', '좋아요']
  def _tokenize(self, texts):
    tokenizer = Okt()

    def _has_preceding_space(text, token, last_position):
      return text[last_position:].startswith(" "+token)

    def _tokenize_a_text(text):
      poses = tokenizer.pos(text)

      tokens = []

      last_position = 0
      for i, pos in enumerate(poses):
        token = pos[0]
        org_token = token
        if i==0 or pos[1]=="Punctuation":
          pass
        elif _has_preceding_space(text, token, last_position):
          last_position += 1
        else:
          token = BERT_PREFIX+token 

        tokens.append(token)
        last_position += len(token)

      return tokens

    # 각 문장별로 토크나이징
    all_tokens = []  
    for text in tqdm(texts):
      all_tokens.extend(_tokenize_a_text(text))

    # 빈도 순으로 정열
    counts = collections.Counter(all_tokens)
    sorted_tokens = sorted(all_tokens, key=counts.get, reverse=True)

    # 단어 중복 삭제
    sorted_tokens = list(OrderedDict.fromkeys(sorted_tokens))

    # Bert의 4개 특수 토큰을 삽입
    sorted_tokens.insert(0, '[PAD]')
    sorted_tokens.insert(1, '[UNK]')
    sorted_tokens.insert(2, '[CLS]')
    sorted_tokens.insert(3, '[SEP]')

    self._tokens = sorted_tokens

  def get_vocab(self):
    return self._tokens
  

In [ ]:
konlply_tokenizer = KonlpyVocabMaker(['견인 회사는 "주권"으로 명명되었다.'])
vocab = konlply_tokenizer.get_vocab()
print(vocab)

konlply_tokenizer = KonlpyVocabMaker(['하늘이 푸른가요? 나는 푸른색이 좋아요'])
vocab = konlply_tokenizer.get_vocab()
print(vocab)

100%|██████████| 1/1 [00:00<00:00, 554.66it/s]


['[PAD]', '[UNK]', '[CLS]', '[SEP]', '"', '견인', '회사', '##는', '##주권', '##으로', '##명명', '##되었다', '.']


100%|██████████| 1/1 [00:00<00:00, 423.80it/s]

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '##이', '하늘', '##푸른가요', '?', '##나', '##는', '##푸른색', '##좋아요']


In [ ]:
print(len(all_sentence))

200000


### 토크나이징 실행

In [ ]:
konlply_tokenizer = KonlpyVocabMaker(all_sentence)

vocab = konlply_tokenizer.get_vocab()
# vocab = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '.', '##를', '##을', '##가', '있다', '##에', '남자', ...]


100%|██████████| 200000/200000 [06:28<00:00, 514.62it/s]


In [ ]:
print(len(vocab))

27879


### vocab 파일 저장

In [ ]:
with open(CUSTOM_VOCAB_FILE, 'w') as f:
  for item in vocab:
    f.write("%s\n" % item)

## Tokenizer 생성

In [ ]:
tokenizer = BertTokenizer(vocab_file=CUSTOM_VOCAB_FILE, do_lower_case=False, model_max_length=SEQ_LENGTH)

In [ ]:
tokenized = tokenizer("다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다.", text_pair="푸른 하늘이 좋아.", max_length=30, padding='max_length')
print("original sentence  :", "다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다.", "푸른 하늘이 좋아.")
print("tokens             :", tokenizer.convert_ids_to_tokens(tokenized['input_ids']))
print("token id           :", tokenized['input_ids'])
print("attention mask     :", tokenized['attention_mask'])
print("token type         :", tokenized['token_type_ids'])

original sentence  : 다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다. 푸른 하늘이 좋아.
tokens             : ['[CLS]', '다', '큰', '남자', '##는', '스파이더맨', '침대', '##보', '##로', '만든', '옷', '##을', '입고', '있다', '.', '[SEP]', '푸른', '하늘', '##이', '[UNK]', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
token id           : [2, 1352, 429, 16, 22, 6536, 1930, 5304, 41, 14129, 94, 7, 12061, 2665, 4, 3, 1062, 1653, 10, 1, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0]
attention mask     : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
token type         : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


![bert_input_architecture](https://user-images.githubusercontent.com/1250095/50039788-8e4e8a00-007b-11e9-9747-8e29fbbea0b3.png)

## x, y 생성


tokernizer 사용 중에 경고 메시지가 많이 뜬다. 억제한다.


In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
def build_model_input(sentences1, sentences2):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  for sentence1, sentence2 in zip(sentences1, sentences2):
    tokenized = tokenizer(sentence1, text_pair=sentence2, max_length=SEQ_LENGTH, padding='max_length')
    # tokenized = {'input_ids': [101, ...], 'token_type_ids': [0, ...], 'attention_mask': [1, ...]}
    input_ids.append(tokenized['input_ids'][:SEQ_LENGTH]) # 버그인지 몰라도 SEQ_LENGTH이상이어도 더 크게 나온다.
    attention_masks.append(tokenized['attention_mask'][:SEQ_LENGTH])
    token_type_ids.append(tokenized['token_type_ids'][:SEQ_LENGTH])

  return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids))


In [ ]:
MAX_DATA_COUNT = 1000000
x = build_model_input(sentences1[:MAX_DATA_COUNT], sentences2[:MAX_DATA_COUNT])
y = labels[:MAX_DATA_COUNT]

x는 다음과 같이 구성됨
```
x = (  token_ids,  attention_masks,  token_types   )
       x[0],       x[1],             [2]
```

<br>

첫번 째 데이터는 
```
   ( token_ids[0],  attention_masks[0], token_types[0] )
 = ( x[0][0],       x[1][0],            x[2][0]  )
```


## train/test 분리

In [ ]:
def split_bert_data(x, y, test_ratio):
  split_index = int(len(y)*(1-test_ratio))
  train_x = (x[0][:split_index], x[1][:split_index], x[2][:split_index])
  test_x  = (x[0][split_index:], x[1][split_index:], x[2][split_index:])
  train_y, test_y = y[:split_index], y[split_index:]

  return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = split_bert_data(x, y, test_ratio=0.2)

In [ ]:
print(sentences1[0], sentences2[0])
print(tokenizer.decode(train_x[0][0][:35]))
print(train_x[0][0][:35])
print(train_x[1][0][:35])
print(train_x[2][0][:35])

다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다. 한 남자가 스파이더맨 침대보를 망토로 사용하여 슈퍼 스파이디가 될 수 있다!
[CLS] 다 큰 남자는 스파이더맨 침대보로 만든 옷을 입고 있다. [SEP] 한 남자가 스파이더맨 침대보를 망토로 사용하여 슈퍼 스파이디가 [UNK] 수 있다!
[    2  1352   429    16    22  6536  1930  5304    41 14129    94     7
 12061  2665     4     3    13    16     8  6536  1930  5304     6  3921
    41 19823   704 14125 14842  4438     8     1  1490  2665  2176]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


# 학습

## 모델 생성

In [ ]:
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout

class TFBertClassifier(tf.keras.Model):
  def __init__(self):
    super(TFBertClassifier, self).__init__()

    self.bert = TFBertModel.from_pretrained(BERT_MODEL_NAME, trainable=True)
    self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
    self.classifier = Dense(3, kernel_initializer=TruncatedNormal(self.bert.config.initializer_range), 
                            name="classifier", activation="softmax")

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=True):

    outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
    pooled_output = outputs[1] 
    v = self.dropout(pooled_output, training=training)
    out = self.classifier(v)

    return out

model = TFBertClassifier()


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

참고로 Bert의 default 설정은 다음과 같다.

In [ ]:
print(model.bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}



In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


## 학습 실행

In [ ]:
history = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
2250/2250 [==============================] - 1089s 484ms/step - loss: 0.8389 - accuracy: 0.6270 - val_loss: 0.8199 - val_accuracy: 0.6499
Epoch 2/5
2250/2250 [==============================] - 1087s 483ms/step - loss: 0.7286 - accuracy: 0.6933 - val_loss: 0.7883 - val_accuracy: 0.6716
Epoch 3/5
2250/2250 [==============================] - 1087s 483ms/step - loss: 0.6451 - accuracy: 0.7385 - val_loss: 0.8211 - val_accuracy: 0.6774
Epoch 4/5
2250/2250 [==============================] - 1087s 483ms/step - loss: 0.5544 - accuracy: 0.7814 - val_loss: 0.8581 - val_accuracy: 0.6790
Epoch 5/5
2250/2250 [==============================] - 1087s 483ms/step - loss: 0.4570 - accuracy: 0.8257 - val_loss: 0.9140 - val_accuracy: 0.6781


In [ ]:
loss, acc = model.evaluate(test_x, test_y, batch_size=32)
print("loss =", loss)
print("acc =", acc)

625/625 [==============================] - 93s 148ms/step - loss: 0.8846 - accuracy: 0.6837
loss = 0.8846092820167542
acc = 0.683650016784668


## 분류 실행

In [ ]:
def do_classify(sentence1, sentence2):
  model_input = build_model_input([sentence1], [sentence2])
  y_ = model.predict(model_input)
  predicted = np.argmax(y_, axis=-1)[0]
  print(sentence1, sentence2, "-->", category_names[predicted], ",score :",y_[0][predicted])

do_classify("나는 왜 그런지 잘 모르겠다.", "나는 그 이유에 관해 확신한다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 그가 왜 학교를 전학했는지 모르겠다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 왜 그런 일이 일어났는지 모르겠어.")

나는 왜 그런지 잘 모르겠다. 나는 그 이유에 관해 확신한다. --> neutral ,score : 0.86810565
나는 왜 그런지 잘 모르겠다. 나는 그가 왜 학교를 전학했는지 모르겠다. --> neutral ,score : 0.7048491
나는 왜 그런지 잘 모르겠다. 나는 왜 그런 일이 일어났는지 모르겠어. --> neutral ,score : 0.510629
